In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/My Drive/수업/Info_Sys_2021/PyTorch

/content/drive/My Drive/수업/Info_Sys_2021/PyTorch


In [ ]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import torch.nn as nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class SingleLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(784, 10) 

    def forward(self, x):
        out=self.layer1(x)
        return out

In [ ]:
model = SingleLayerModel().to(device)

model parameters는 이미 initialize 되어 있다.

In [ ]:
for name in model.parameters():
    print(name)

Parameter containing:
tensor([[-2.9863e-02, -6.3304e-04, -6.9076e-03,  ..., -1.8779e-02,
          3.0876e-02, -3.1737e-02],
        [ 6.7273e-03,  2.0005e-02, -2.4505e-02,  ..., -2.7296e-02,
         -3.8136e-03,  1.1463e-03],
        [-1.7847e-02, -2.9822e-02,  7.4012e-05,  ..., -3.2111e-02,
          2.6378e-02,  3.2280e-02],
        ...,
        [-1.3945e-02,  3.2514e-02,  2.6534e-03,  ..., -3.2780e-02,
         -1.6748e-02, -5.4733e-03],
        [ 1.0081e-02, -1.2429e-02, -1.0554e-02,  ..., -2.0057e-02,
         -9.3248e-04,  8.3561e-03],
        [-6.6728e-03,  7.1139e-03,  2.5636e-02,  ...,  1.9851e-02,
         -1.8774e-02, -3.2610e-02]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0229, -0.0224,  0.0170, -0.0287, -0.0111,  0.0345,  0.0230,  0.0343,
         0.0262, -0.0135], device='cuda:0', requires_grad=True)


In [ ]:
# define cost/loss & optimizer
#criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.616458118
Epoch: 0002 cost = 0.344646066
Epoch: 0003 cost = 0.308612585
Epoch: 0004 cost = 0.291643143
Epoch: 0005 cost = 0.281719834
Epoch: 0006 cost = 0.274483085
Epoch: 0007 cost = 0.269206583
Epoch: 0008 cost = 0.265227377
Epoch: 0009 cost = 0.262127012
Epoch: 0010 cost = 0.258987486
Epoch: 0011 cost = 0.256578743
Epoch: 0012 cost = 0.254623771
Epoch: 0013 cost = 0.252491772
Epoch: 0014 cost = 0.251237988
Epoch: 0015 cost = 0.249651745
Learning finished


In [ ]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.902999997138977
Label:  8
Prediction:  3


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
